# Web scraping

A la hora de recoger información de terceros, lo ideal es consultar una API, porque:

* Las respuestas contienen información estructurada
* En general, el propio servicio nos facilitará documentación sobre la estrucutra del API, (endpoints), cómo realizar peticiones y qué tipo de información podemos solicitar.

Sin embargo, muchas veces encontramos información en páginas web (en formato [HTML](https://es.wikipedia.org/wiki/HTML)) que nos gustaría obtener, pero sin un API disponible.

Estas páginas `HTML` tienen cierta estructura, aunque con ciertos contras:

* Es más compleja, puede tener muchos niveles de anidamiento
* Es inestable. Están diseñadas para que se vean bien desde el explorador, no para guardar una estructura de consulta. Puede verse alterada por la incorporación de nuevos elementos visuales u otros motivos.
* Puede ser modificada por código cliente (javascript) en diferentes momentos: al cargar la página, al interaccionar con algún elemento, ...
* Cada día son más las webs protegidas contra la extracción de datos.

### Ejercicio

Desde tu explorador, consulta el código fuente de una página de tu interés. Por ejemplo, para hacerlo en chrome:

* Accede a la página, p.e. [esta](https://es.wikipedia.org/wiki/HTML).
* Haz click derecho y pulsa `View page source`. Otra opción es pulsar `Inspect`, que además abrirá las herramientas de desarrollador de Chrome, muy útiles para navegar por la estructura de la página.

## Scraping de elementos html

La librería que vamos a utilizar es [Beautiful Soup](https://pypi.org/project/beautifulsoup4/). Nos permite buscar elementos y navegar por la estructura del html fácilmente.

Vemos dos ejemplos, uno sobre milanuncios y otro sobre spotahome, por si nos _banean_.

### milanuncios

Imaginemos que queremos comparar precios de un determinado modelo de motocicleta de segunda mano. P.e. con [esta búsqueda](https://www.milanuncios.com/motos-de-carretera/duke-390.htm) en milanuncios.

La mayor parte de las webs con contenido interesante (que hacen negocio gracias a su contenido) intentan protegerlas para evitar que les hagan scraping. Hay varias formas de simular que nuestro script es humano en lugar de un bot, algunas más básicas y otras más complejas. Por ahora, vamos a sobrescribir nuestro _user agent_. Es una cabecera que va en las peticiones diciendo quiénes somos (p.e. qué tipo de explorador usamos). Por defecto, la librería `requests` que vamos a usar, avisa que somos un bot. Vamos a sobrescribir esta cabecera para _disimular_ un poco. Podemos copiar uno popular de (aquí)[https://developers.whatismybrowser.com/useragents/explore/software_type_specific/web-browser/].

In [376]:
import requests
from bs4 import BeautifulSoup

In [377]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

Ahora, nos descargamos el html con `requests`.

In [378]:
page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm', headers=headers)
page

<Response [200]>

Podemos ver el contenido examinando la propiedad `content`.

In [379]:
page.content

b'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="es" lang="es">\n<head><title>MIL ANUNCIOS.COM - Duke 390. Motos de carretera de ocasion duke 390: Aprilia, BMW, Gagiva, Dervi, Honda, Yamaha, Kawasaki, Suzuki.</title>\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\n    <meta name="robots" content="noarchive">\n        <meta name="description" content="Compra venta de motos de carretera de ocasi\xf3n duke 390. Todas las marcas: Aprilia, BMW, Cagiva, Derbi, Honda, Yamaha, Kawasaki, Suzuki">\n<link rel="shortcut icon" href="https://static.milanuncios.com/202012071144-master/favicon.ico">    <link rel="stylesheet" type="text/css" href="https://static.milanuncios.com/202012071144-master/css/estilos.css">\n    <link rel="canonical" href="https://www.milanuncios.com/motos-de-carretera/duke-390.htm" />\n<script src="//c.dcdn.es/borostcf/plugins/BorosTcfAdevintaPlugin.pro.js"></script>\n<script src="//c.dcdn.es/openads/milanuncios/MilanunciosOpen

Este contenido es solo texto, no tiene estructura. Aún no podemos hacer búsquedas ni navegar por él.
 
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Para hacerlo, creamos una instancia de `Beautiful Soup` y lo parseamos

In [380]:
soup = BeautifulSoup(page.content, 'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="es" xml:lang="es" xmlns="http://www.w3.org/1999/xhtml">
<head><title>MIL ANUNCIOS.COM - Duke 390. Motos de carretera de ocasion duke 390: Aprilia, BMW, Gagiva, Dervi, Honda, Yamaha, Kawasaki, Suzuki.</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="noarchive" name="robots"/>
<meta content="Compra venta de motos de carretera de ocasión duke 390. Todas las marcas: Aprilia, BMW, Cagiva, Derbi, Honda, Yamaha, Kawasaki, Suzuki" name="description"/>
<link href="https://static.milanuncios.com/202012071144-master/favicon.ico" rel="shortcut icon"/> <link href="https://static.milanuncios.com/202012071144-master/css/estilos.css" rel="stylesheet" type="text/css"/>
<link href="https://www.milanuncios.com/motos-de-carretera/duke-390.htm" rel="canonical">
<script src="//c.dcdn.es/borostcf/plugins/BorosTcfAdevintaPlugin.pro.js"></script>
<script defer="" src="//c.dcdn.es/openads/milanuncios/MilanunciosOpenAdsClient.pro.js"></scrip

### Extracción de los diferentes de componentes del fichero HTML. Navigando por la estructura de datos

In [6]:
soup.title

<title>MIL ANUNCIOS.COM - Duke 390. Motos de carretera de ocasion duke 390: Aprilia, BMW, Gagiva, Dervi, Honda, Yamaha, Kawasaki, Suzuki.</title>

In [7]:
soup.title.name

'title'

### EJERCICIO.
Consulta la documentación de `Beautiful Soup` y busca como extraer la cadena de texto que aparece contenido en la etiqueta HTML `Title`

In [8]:
# Respuesta
soup.title.string

'MIL ANUNCIOS.COM - Duke 390. Motos de carretera de ocasion duke 390: Aprilia, BMW, Gagiva, Dervi, Honda, Yamaha, Kawasaki, Suzuki.'

In [9]:
soup.a

<a href="/" title="segunda mano">
<div></div>
</a>

### EJERCICIO.
Consultando la documentación de `Beautiful Soup`, encuentra la forma de visualizar todas las etiquetas de referencias (enlaces "a") que aparecen en el HTML de la página descargada.

In [10]:
# Respuesta
soup.find_all('a')

[<a href="/" title="segunda mano">
 <div></div>
 </a>,
 <a class="rutenl" href="https://www.milanuncios.com">Milanuncios</a>,
 <a class="rutenl" href="/motor/" title="Motor">Motor</a>,
 <a class="rutenl" href="/motos-de-segunda-mano/" title="Motos de segunda mano">Motos</a>,
 <a class="rutenl" href="/motos-de-carretera/" title="Motos de carretera">Motos de carretera</a>,
 <a class="fillDiv" href="javascript:cabu()" id="vamos" tabindex="85">BUSCAR
         </a>,
 <a href="javascript:cabu()" style="color: #30832F;">BUSCA</a>,
 <a class="aditem-detail-title" href="ktm-duke-390-abs-374733585.htm" target="_blank">KTM - <b class="sub2">DUKE</b> <b class="sub3">390</b> ABS</a>,
 <a href="javascript:cpr(2)">
 <div class="pillDiv pillSellerTypePriv">Particular</div>
 </a>,
 <a href="ktm-duke-390-abs-374733585.htm#fotos" target="_blank">Ver fotos</a>,
 <a href="javascript:var urlFotos_374733585 = [];urlFotos_374733585['0'] = 'https://img.milanuncios.com/fg/3747/33/374733585_1.jpg?VersionId=ARV53

### Ejercicio evaluación (PARA REALIZAR EN CASA)
Extrae todas las cadenas que comienzan con `'https://`. Tienes que extraer desde `'https://` hasta el final. Ej.: 'https://img.milanuncios.com/fg/3749/86/374986816_1.jpg?VersionId=eUgig6FRM4L8.a8VbwgAsPfHUxDoCudE'

In [381]:
# Respuesta
for link in soup.find_all('a'):    
    href = str(link.get('href'))    
    if(href.startswith('https')):
        print(href)

https://www.milanuncios.com
https://ayuda.milanuncios.com/hc/es
https://play.google.com/store/apps/details?id=com.muba.anuncios&hl=es
https://itunes.apple.com/es/app/milanuncios-tablon-anuncios/id967185651?mt=8
https://www.facebook.com/milanuncios
https://twitter.com/milanuncios
https://www.instagram.com/milanuncios/
https://www.youtube.com/channel/UCw6Jn6QF5L9NXYkr_XNFMCQ
https://ayuda.milanuncios.com/hc/es
https://www.milanuncios.com/prensa/?ipromo=home-salaprensa
https://www.adevinta.com/es/spain
https://www.fotocasa.es
https://www.habitaclia.com
https://www.infojobs.net
https://www.coches.net
https://motos.coches.net
https://www.vibbo.com


### Ejercicio
Busca las etiquetas `text/css` del fichero.

In [382]:
# Respuesta
#soup.findAll('link', type='text/css')
soup.find(type='text/css')

<link href="https://static.milanuncios.com/202012071144-master/css/estilos.css" rel="stylesheet" type="text/css"/>

In [383]:
# Refinando la búsqueda para encontrar los precios que es lo que buscamos
soup.find_all(class_='aditem-price')

[<div class="aditem-price">2.700<sup>€</sup></div>,
 <div class="aditem-price">2.250<sup>€</sup></div>,
 <div class="aditem-price">1.000<sup>€</sup></div>,
 <div class="aditem-price">3.850<sup>€</sup></div>,
 <div class="aditem-price">3.000<sup>€</sup></div>,
 <div class="aditem-price">4.200<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">2.800<sup>€</sup></div>,
 <div class="aditem-price">3.900<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">3.000<sup>€</sup></div>,
 <div class="aditem-price">4.500<sup>€</sup></div>,
 <div class="aditem-price">4.099<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">2.900<sup>€</sup></div>,
 <div class="aditem-price">4.900<sup>€</sup></div>,
 <div class="aditem-price">3.900<sup>€</sup></div>,
 <div class="aditem-price">3.950<sup>€</sup></div>,
 <div class="aditem-price">3.399<sup>€</sup></div>,
 <div class=

Sobre esto, podemos hacer búsquedas con `find` y `find_all` (o `select_one` y `select` si prefieres utilizar [selectores css](https://en.wikipedia.org/wiki/Cascading_Style_Sheets#Selector)). Sobre nuestro ejemplo, vamos a buscar todos los precios. Examinando el código fuente, vemos que son etiquetas `div` con clase `aditem-price`.

Vemos que los precios se ordanizan en etiquetas `<div>` y `class="aditem-price"`, tal que `<div class="aditem-price">4.500<sup>&euro`

In [384]:
div_precios = soup.find_all('div', class_='aditem-price')
div_precios

[<div class="aditem-price">2.700<sup>€</sup></div>,
 <div class="aditem-price">2.250<sup>€</sup></div>,
 <div class="aditem-price">1.000<sup>€</sup></div>,
 <div class="aditem-price">3.850<sup>€</sup></div>,
 <div class="aditem-price">3.000<sup>€</sup></div>,
 <div class="aditem-price">4.200<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">2.800<sup>€</sup></div>,
 <div class="aditem-price">3.900<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">3.000<sup>€</sup></div>,
 <div class="aditem-price">4.500<sup>€</sup></div>,
 <div class="aditem-price">4.099<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">2.900<sup>€</sup></div>,
 <div class="aditem-price">4.900<sup>€</sup></div>,
 <div class="aditem-price">3.900<sup>€</sup></div>,
 <div class="aditem-price">3.950<sup>€</sup></div>,
 <div class="aditem-price">3.399<sup>€</sup></div>,
 <div class=

`find_all` devuelve una lista de elementos. Sobre ellos, podemos hacer:

`children` para sacar el listado de todos los hijos.

In [385]:
list(div_precios[0].children)

['2.700', <sup>€</sup>]

`get_text()` para sacar el texto de todos los hijos

In [386]:
div_precios[0].get_text()

'2.700€'

Por tanto, para sacar el listado de todos los precios podemos ejecutar la siguiente instrucción:

In [387]:
[list(div_precio.children)[0] for div_precio in div_precios]

['2.700',
 '2.250',
 '1.000',
 '3.850',
 '3.000',
 '4.200',
 '3.500',
 '2.800',
 '3.900',
 '3.500',
 '3.000',
 '4.500',
 '4.099',
 '3.500',
 '2.900',
 '4.900',
 '3.900',
 '3.950',
 '3.399',
 '3.300',
 '4.799',
 '3.600',
 '3.800',
 '4.200',
 '4.690',
 '4.290',
 '3.100',
 '3.650',
 '2.000',
 '3.500']

Tienes más funciones útiles con pequeños ejemplos [aquí](http://akul.me/blog/2016/beautifulsoup-cheatsheet/)

### Ejercicio evaluación (PARA CASA)

Crea un dataframe de pandas en el que cada fila sea un anuncio y tenga como columnas información relevante: precio, kilómetros, año, cilindrada, texto del anuncio, ...

In [388]:
import pandas as pd

def getTextByClassName(tag, className):       
    if(tag.select(className)):
        return tag.select(className)[0].get_text()
    else:
        return ''

div_anuncios = soup.find_all("div", id="cuerpo")
container = div_anuncios[0].find_all("div",  {"class": ["aditem", "ProfesionalCardTestABClass"]})

titulos = []
precios = []
regiones = []
ccs = []
anns = []
kms = []
gas = []

i = 0
for c in container:            
    titulos.append(getTextByClassName(c, '.aditem-detail-title'))    
    precios.append(getTextByClassName(c, '.aditem-price'))    
    regiones.append(getTextByClassName(c, '.list-location-region'))
    if(c.select(".inmo-attributes")):
        for a in c.select(".inmo-attributes"):
            ccs.append(getTextByClassName(a, ".cc"))            
            anns.append(getTextByClassName(a, ".ano"))            
            kms.append(getTextByClassName(a, ".kms"))            
            gas.append(getTextByClassName(a, ".gas"))    

d = {'Titulo': titulos, 'Region': regiones, 'Precio': precios, 'Cilindrada': ccs, 'Año': anns, 'Kilometros': kms, 'Combustible': gas}
df = pd.DataFrame(data=d)

df

,Titulo,Region,Precio,Cilindrada,Año,Kilometros,Combustible
0,KTM - DUKE 390,Ademuz,2.700€,,,,
1,KTM - DUKE 390,Valdemoro,2.250€,390 cc,año 2016,49.000 kms,
2,SE COMPRAN MOTOCICLETAS HONDA YAMAHA KAW,barcelona,1.000€,,,,
3,KTM - DUKE 390 ABS,Guadalupe,3.850€,400 cc,año 2016,2.000 kms,gasolina
4,SUZUKI - GSR,A Coruña,3.000€,600 cc,año 2009,41.600 kms,
5,KTM - 390 DUKE,Santander,4.200€,390 cc,año 2017,9.600 kms,
6,KTM - KTM DUKE 390,Fuente del Maestre,3.500€,390 cc,año 2014,14.956 kms,
7,YAMAHA - FZ6N,Roquetas de Mar,2.800€,600 cc,año 2006,20.000 kms,
8,KTM - 390,paiporta,3.900€,373 cc,año 2019,9.100 kms,gasolina
9,KTM - DUKE 390 ABS BLACK,Roquetas de Mar,3.500€,390 cc,año 2014,26.000 kms,gasolina


## Scraping de tablas

A menudo, la información que nos interesa descargar está en tablas y nuestro objetivo es importarlas en tablas de Pandas. Esta conversión suele exigir la manipulación del texto, números y fechas contenidas en la tabla original, lo que nos obligará a repasar cómo realizar esas operaciones y aplicarlas a filas y columnas de las tablas.

La estructura que suelen tener la tablas en `html` es:

```
<table>
    <thead>
        <tr>
            <th>Columna A</th>
            <th>Columna B</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>A1</td>
            <td>B1</td>
        </tr>
        <tr>
            <td>A2</td>
            <td>B2</td>
        </tr>
    </tbody>
</table>   
```

Necesitaremos los siguientes módulos además de `requests` y `BeautifulSoup` importados anteriormente:

In [28]:
import pandas as pd
import re

Primero, hacemos una petición para descargar la página de interés (que contiene las cotizaciones de las acciones del IBEX 35 en tiempo _casi_ real).

In [201]:
base_url = "https://www.eleconomista.es/indice/IBEX-35"
res = requests.get(base_url)
contenido = res.content
print(contenido)

b'<!DOCTYPE html>\r\n\t\t\t\t\t\t\t<html lang="es">\r\n\t\t\t\t\t<head><title>IBEX 35 - Cotizaciones en tiempo real</title>\r\n\t\t\t\t\t\t<meta charset="utf-8" />\r\n\t\t\t\t\t\t<meta http-equiv="X-UA-Compatible" content="IE=Edge" /><meta property="og:title" name="title" content="IBEX 35 - Cotizaciones en tiempo real" />\n <!--[if lt IE 9]>\r\n\t\t\t\t\t\t<link rel="stylesheet" href="/css/ie.css">\r\n\t\t\t\t\t<![endif]-->\n<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1"><!-- Inicio de notificacion de consentimiento de cookies de Didomi -->\r\n                \r\n                \r\n                <script type="text/javascript" id="spcloader" src="https://s03.s3c.es/js/didomicmpconfig_v2_1.js?v=2_1"></script>\r\n\r\n\r\n                <!-- Final de notificacion de consentimiento de cookies de Didomi --><script>\r\n\t\tfunction getModoWEB() {\r\n\t\t\tvar width = screen.width;\r\n\t\t\tif (width <= 768) {\r\n\t\t\t\treturn "movil";\r\n\t\t\t}\r\n\

La siguiente línea procesa el HTML de la página que hemos descargado:

In [202]:
soup = BeautifulSoup(contenido, "html.parser")

Una vez procesado el HTML, es posible buscar elementos dentro de él. En particular, podemos buscar los elementos de tipo `table`, es decir, tablas.

In [203]:
tablas = soup.find_all('table')
print(tablas)

[<table class="table tableFlex table-striped footable footable-1 breakpoint-lg" data-expand-first="false" data-show-toggle="true" data-toggle-column="last"><thead><tr class="footable-header"><th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th><th>Precio</th><th data-breakpoints="" data-title=""></th><th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th><th data-breakpoints="xs">Var. (€)</th><th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th><th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th><th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th><th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th><th class="footable-last-visible" data-breakpoints="sm xs

El objeto `tablas` contiene todas las tablas presentes en la página. Hay que tener cuidado con dichas tablas porque muchas páginas utilizan elementos de tipo `table` para estructurar el contenido. Por eso, en algunas páginas, aunque parezca haber una única tabla, puede haber otras con una información no interesante que toca descartar.

In [204]:
len(tablas)

3

Podemos extraer las filas de todas estas tablas

In [37]:
tablas

[<table class="table tableFlex table-striped footable footable-1 breakpoint-lg" data-expand-first="false" data-show-toggle="true" data-toggle-column="last"><thead><tr class="footable-header"><th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th><th>Precio</th><th data-breakpoints="" data-title=""></th><th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th><th data-breakpoints="xs">Var. (€)</th><th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th><th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th><th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th><th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th><th class="footable-last-visible" data-breakpoints="sm xs

### Ejercicio 1
 
Mediante el uso de bucles, todos los elementos de tabla `tr`. Puedes usar `find_all('tr')` para encontrar todos los `tr` de la página.
 
Imprime los resultados en pantalla.

In [207]:
# Respuesta 
#for tabla in tablas:
#    for x in tabla.find_all('tr'):
#        print(x, end="\n")

[list(table.children) for table in soup.find_all('tr')]
#[x for tabla in tablas for x in tabla.find_all('tr')]

[[<th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th>,
  <th>Precio</th>,
  <th data-breakpoints="" data-title=""></th>,
  <th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th>,
  <th data-breakpoints="xs">Var. (€)</th>,
  <th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th>,
  <th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th>,
  <th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th>,
  <th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th>,
  <th class="footable-last-visible" data-breakpoints="sm xs" data-title="Hora"> Hora </th>],
 [<td class="footable-first-visible" itemscope="" itemtype="http://schema.org/SiteNavigationElement"><a href="/empresa/

### Ejercicio 2
 
A partir del ejercicio 1, compleméntalo con los siguientes pasos:
* Crea una lista vacía al inicio
* Usando el método `append()` inserta las filas de las tabla en formato lista/array

In [208]:
# Respuesta
listaTablas = []

for tabla in tablas:
    for x in tabla.find_all('tr'):
        listaTablas.append(x)
print(listaTablas)

[<tr class="footable-header"><th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th><th>Precio</th><th data-breakpoints="" data-title=""></th><th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th><th data-breakpoints="xs">Var. (€)</th><th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th><th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th><th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th><th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th><th class="footable-last-visible" data-breakpoints="sm xs" data-title="Hora"> Hora </th></tr>, <tr><td class="footable-first-visible" itemscope="" itemtype="http://schema.org/SiteNavigationElement"><a href="/empresa/ACC

In [210]:
# Otra forma de hacerlo más directa
# Comprehensions
lineas = [x for tabla in tablas for x in tabla.find_all('tr')]

In [58]:
lineas

[<tr class="footable-header"><th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th><th>Precio</th><th data-breakpoints="" data-title=""></th><th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th><th data-breakpoints="xs">Var. (€)</th><th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th><th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th><th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th><th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th><th class="footable-last-visible" data-breakpoints="sm xs" data-title="Hora"> Hora </th></tr>,
 <tr><td class="footable-first-visible" itemscope="" itemtype="http://schema.org/SiteNavigationElement"><a href="/empresa/AC

### Extraer el texto (datos) de las tablas

In [211]:
lista_acciones = []
temp_lista = []

for fila in lineas:
    for linea in fila.find_all('td'):
        temp_lista.append(linea.text) # El método text nos devuelve los datos/texto dentro del código html
    print(temp_lista)
    lista_acciones.append(temp_lista)
    temp_lista = []

lista_acciones[0:3]

[]
['ACCIONA', '107,00', '', '+1,42%', '1,50', '16.944.738,40', '0,00', '29,32', '2,14%', '4/12']
['ACERINOX', '9,01', '', '+4,79%', '0,41', '13.302.305,46', '0,00', '40,76', '5,39%', '4/12']
['ACS', '27,52', '', '+3,34%', '0,89', '21.220.632,61', '0,00', '13,34', '5,87%', '4/12']
['AENA', '141,80', '', '-0,14%', '-0,20', '30.522.709,90', '0,00', '14,96', '0,00%', '4/12']
['ALMIRALL', '10,70', '', '+0,38%', '0,04', '2.787.644,04', '0,00', '23,36', '1,78%', '4/12']
['AMADEUS', '64,00', '', '+2,70%', '1,68', '94.051.072,36', '0,00', '20,45', '0,00%', '4/12']
['ARCELORMITTAL', '17,44', '', '+2,93%', '0,50', '14.805.115,98', '0,00', '0,00', '0,00%', '4/12']
['BANKIA', '1,53', '', '+0,86%', '0,01', '9.025.187,14', '0,00', '53,16', '0,00%', '4/12']
['BANKINTER', '4,48', '', '+3,51%', '0,15', '41.437.145,64', '0,00', '14,73', '1,92%', '4/12']
['BBVA', '4,30', '', '+5,54%', '0,23', '226.751.207,40', '0,00', '10,15', '0,76%', '4/12']
[]
['CAIXABANK', '2,25', '', '+0,71%', '0,02', '36.172.917,94

[[],
 ['ACCIONA',
  '107,00',
  '',
  '+1,42%',
  '1,50',
  '16.944.738,40',
  '0,00',
  '29,32',
  '2,14%',
  '4/12'],
 ['ACERINOX',
  '9,01',
  '',
  '+4,79%',
  '0,41',
  '13.302.305,46',
  '0,00',
  '40,76',
  '5,39%',
  '4/12']]

### Otra forma, más compacta de conseguir el mismo resultado

para luego extraer los contenidos de cada fila individualmente haciendo

In [213]:
datos = [[x.text for x  in linea.find_all('td')] for linea in lineas]

Podemos inspeccionar parte del objeto resultante:

In [214]:
datos[0:3]

[[],
 ['ACCIONA',
  '107,00',
  '',
  '+1,42%',
  '1,50',
  '16.944.738,40',
  '0,00',
  '29,32',
  '2,14%',
  '4/12'],
 ['ACERINOX',
  '9,01',
  '',
  '+4,79%',
  '0,41',
  '13.302.305,46',
  '0,00',
  '40,76',
  '5,39%',
  '4/12']]

### Ejercicio 2
 
Recorre la lista creada en el apartado anterior `lista_acciones`, en mi caso, y:
* Elimina las filas sin contenido.
* Guarda la lista limpia en una nueva lista. Por ejemplo `lista_acciones_limpia`

In [217]:
# Respuesta
lista_acciones_limpia = []

for lista in lista_acciones:
    if(len(lista)):
        lista_acciones_limpia.append(lista)

lista_acciones_limpia

[['ACCIONA',
  '107,00',
  '',
  '+1,42%',
  '1,50',
  '16.944.738,40',
  '0,00',
  '29,32',
  '2,14%',
  '4/12'],
 ['ACERINOX',
  '9,01',
  '',
  '+4,79%',
  '0,41',
  '13.302.305,46',
  '0,00',
  '40,76',
  '5,39%',
  '4/12'],
 ['ACS',
  '27,52',
  '',
  '+3,34%',
  '0,89',
  '21.220.632,61',
  '0,00',
  '13,34',
  '5,87%',
  '4/12'],
 ['AENA',
  '141,80',
  '',
  '-0,14%',
  '-0,20',
  '30.522.709,90',
  '0,00',
  '14,96',
  '0,00%',
  '4/12'],
 ['ALMIRALL',
  '10,70',
  '',
  '+0,38%',
  '0,04',
  '2.787.644,04',
  '0,00',
  '23,36',
  '1,78%',
  '4/12'],
 ['AMADEUS',
  '64,00',
  '',
  '+2,70%',
  '1,68',
  '94.051.072,36',
  '0,00',
  '20,45',
  '0,00%',
  '4/12'],
 ['ARCELORMITTAL',
  '17,44',
  '',
  '+2,93%',
  '0,50',
  '14.805.115,98',
  '0,00',
  '0,00',
  '0,00%',
  '4/12'],
 ['BANKIA',
  '1,53',
  '',
  '+0,86%',
  '0,01',
  '9.025.187,14',
  '0,00',
  '53,16',
  '0,00%',
  '4/12'],
 ['BANKINTER',
  '4,48',
  '',
  '+3,51%',
  '0,15',
  '41.437.145,64',
  '0,00',
  '14,73

Vemos que hay filas que contienen la información de interés junto con otras que contienen cabeceras y otra información irrelevante. En general, la situación puede ser más complicada y se hace necesario estudiar el objeto `tablas` para seleccionar la de interés.

En nuestro caso, podemos filtrar las líneas menos relevantes así:

In [218]:
datos = [x for x in datos if len(x) > 0]

### Ejercicio evaluación 1
 
Guarda los datos ya limpios en un dataframe de Pandas con nombre `df_IBEX35`

In [333]:
# Respuesta
df_IBEX35 = pd.DataFrame(data=lista_acciones_limpia)

Finalmente, podemos crear una tabla de Pandas:

In [334]:
# Respuesta
df_IBEX35

,0,1,2,3,4,5,6,7,8,9
0,ACCIONA,"107,00",,"+1,42%","1,50","16.944.738,40","0,00","29,32","2,14%",4/12
1,ACERINOX,"9,01",,"+4,79%","0,41","13.302.305,46","0,00","40,76","5,39%",4/12
2,ACS,"27,52",,"+3,34%","0,89","21.220.632,61","0,00","13,34","5,87%",4/12
3,AENA,"141,80",,"-0,14%","-0,20","30.522.709,90","0,00","14,96","0,00%",4/12
4,ALMIRALL,"10,70",,"+0,38%","0,04","2.787.644,04","0,00","23,36","1,78%",4/12
5,AMADEUS,"64,00",,"+2,70%","1,68","94.051.072,36","0,00","20,45","0,00%",4/12
6,ARCELORMITTAL,"17,44",,"+2,93%","0,50","14.805.115,98","0,00","0,00","0,00%",4/12
7,BANKIA,"1,53",,"+0,86%","0,01","9.025.187,14","0,00","53,16","0,00%",4/12
8,BANKINTER,"4,48",,"+3,51%","0,15","41.437.145,64","0,00","14,73","1,92%",4/12
9,BBVA,"4,30",,"+5,54%","0,23","226.751.207,40","0,00","10,15","0,76%",4/12


#### Ejercicio evaluación 2

Usa los elementos `th` de la primera fila de las tablas para extraer nombres para las columnas de la tabla. 

In [335]:
# Respuesta
th = soup.find_all("table", limit=1)

for h in th[0].find_all("th"):
    print(h.getText())

Nombre
Precio

Var. (%)
Var. (€)
Volumen (€)
Capitalización(1)
PER
Rent. /Div.
 Hora 


### Ejercicio evaluación 3
 
En el ejecicio evaluación 2:
* Guarda los resultado en forma de lista/array
* Modifica las columnas del dataframe df_IBEX35 por el nombre adecuado, obtenidos en este ejercicio

In [336]:
# Respuesta
th = soup.find_all("table", limit=1)
headers = []

for h in th[0].find_all("th"):
    headers.append(h.getText())

#df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=['a', 'b', 'c'])
df_IBEX35 = pd.DataFrame(data=lista_acciones_limpia, columns=headers)
df_IBEX35

,Nombre,Precio,,Var. (%),Var. (€),Volumen (€),Capitalización(1),PER,Rent. /Div.,Hora
0,ACCIONA,"107,00",,"+1,42%","1,50","16.944.738,40","0,00","29,32","2,14%",4/12
1,ACERINOX,"9,01",,"+4,79%","0,41","13.302.305,46","0,00","40,76","5,39%",4/12
2,ACS,"27,52",,"+3,34%","0,89","21.220.632,61","0,00","13,34","5,87%",4/12
3,AENA,"141,80",,"-0,14%","-0,20","30.522.709,90","0,00","14,96","0,00%",4/12
4,ALMIRALL,"10,70",,"+0,38%","0,04","2.787.644,04","0,00","23,36","1,78%",4/12
5,AMADEUS,"64,00",,"+2,70%","1,68","94.051.072,36","0,00","20,45","0,00%",4/12
6,ARCELORMITTAL,"17,44",,"+2,93%","0,50","14.805.115,98","0,00","0,00","0,00%",4/12
7,BANKIA,"1,53",,"+0,86%","0,01","9.025.187,14","0,00","53,16","0,00%",4/12
8,BANKINTER,"4,48",,"+3,51%","0,15","41.437.145,64","0,00","14,73","1,92%",4/12
9,BBVA,"4,30",,"+5,54%","0,23","226.751.207,40","0,00","10,15","0,76%",4/12


#### Ejercicio

Elimina las columnas irrelevantes y cambia los nombres de las columnas por otros breves y sin caracteres extraños o que dificulten el posproceso.

In [337]:
#df_IBEX35.drop(df_IBEX35.columns[2], axis=1, inplace=True)
df_IBEX35 = df_IBEX35.rename(columns={"Var. (%)": "VariacionPorcentaje", 
                                      "Var. (€)": "VariacionEuros", 
                                      "Volumen (€)": "Volumen", 
                                      "Capitalización(1)": "Capitalizacion", 
                                      "Rent. /Div.": "RentDiv",
                                      " Hora ": "Hora"})

#### Ejercicio

Cambia el formato de las columnas adecuadamente: convierte a numéricas, etc., las columnas que lo requieran.

In [374]:
from datetime import date

# Converting Precio from string to float
df_IBEX35['Precio'] = df_IBEX35['Precio'].map(lambda x: x.replace(',','.'))
df_IBEX35['Precio'] = df_IBEX35['Precio'].astype(float)

# Converting VariacionEuros from string to float
df_IBEX35['VariacionEuros'] = df_IBEX35['VariacionEuros'].map(lambda x: x.replace(',','.'))
df_IBEX35['VariacionEuros'] = df_IBEX35['VariacionEuros'].astype(float)

# Converting Volumen from string to float
df_IBEX35['Volumen'] = df_IBEX35['Volumen'].map(lambda x: x.replace('.',''))
df_IBEX35['Volumen'] = df_IBEX35['Volumen'].map(lambda x: x.replace(',','.'))
df_IBEX35['Volumen'] = df_IBEX35['Volumen'].astype(float)

# Converting Capitalizacion from string to float
df_IBEX35['Capitalizacion'] = df_IBEX35['Capitalizacion'].map(lambda x: x.replace(',','.'))
df_IBEX35['Capitalizacion'] = df_IBEX35['Capitalizacion'].astype(float)

# Converting PER from string to float
df_IBEX35['PER'] = df_IBEX35['PER'].map(lambda x: x.replace(',','.'))
df_IBEX35['PER'] = df_IBEX35['PER'].astype(float)

# Converting RentDiv from string to float
df_IBEX35['RentDiv'] = df_IBEX35['RentDiv'].map(lambda x: x.replace(',','.'))
df_IBEX35['RentDiv'] = df_IBEX35['RentDiv'].map(lambda x: x.strip('%'))
df_IBEX35['RentDiv'] = df_IBEX35['RentDiv'].astype(float)
df_IBEX35['RentDiv'] = df_IBEX35['RentDiv'].map(lambda x: x/100.0)

# Converting Hora from string to date
df_IBEX35['Hora'] = df_IBEX35['Hora'].map(lambda x: date(2020, int(x.split('/')[0]), int(x.split('/')[1])))
pd.to_datetime(df_IBEX35['Hora'])

# Converting VariacionPorcentaje from string to float
df_IBEX35['VariacionPorcentaje'] = df_IBEX35['VariacionPorcentaje'].map(lambda x: x.strip('%'))
df_IBEX35['VariacionPorcentaje'] = df_IBEX35['VariacionPorcentaje'].map(lambda x: x.strip('+'))
df_IBEX35['VariacionPorcentaje'] = df_IBEX35['VariacionPorcentaje'].map(lambda x: x.replace(',','.'))
df_IBEX35['VariacionPorcentaje'] = df_IBEX35['VariacionPorcentaje'].astype(float)
df_IBEX35['VariacionPorcentaje'] = df_IBEX35['VariacionPorcentaje'].map(lambda x: x/100.0)

df_IBEX35

2020-04-12
<class 'datetime.date'>


,Nombre,Precio,,VariacionPorcentaje,VariacionEuros,Volumen,Capitalizacion,PER,RentDiv,Hora
0,ACCIONA,107.00,,0.0142,1.50,1.694474e+07,0.0,29.32,0.0214,2020-04-12
1,ACERINOX,9.01,,0.0479,0.41,1.330231e+07,0.0,40.76,0.0539,2020-04-12
2,ACS,27.52,,0.0334,0.89,2.122063e+07,0.0,13.34,0.0587,2020-04-12
3,AENA,141.80,,-0.0014,-0.20,3.052271e+07,0.0,14.96,0.0000,2020-04-12
4,ALMIRALL,10.70,,0.0038,0.04,2.787644e+06,0.0,23.36,0.0178,2020-04-12
5,AMADEUS,64.00,,0.0270,1.68,9.405107e+07,0.0,20.45,0.0000,2020-04-12
6,ARCELORMITTAL,17.44,,0.0293,0.50,1.480512e+07,0.0,0.00,0.0000,2020-04-12
7,BANKIA,1.53,,0.0086,0.01,9.025187e+06,0.0,53.16,0.0000,2020-04-12
8,BANKINTER,4.48,,0.0351,0.15,4.143715e+07,0.0,14.73,0.0192,2020-04-12
9,BBVA,4.30,,0.0554,0.23,2.267512e+08,0.0,10.15,0.0076,2020-04-12


## Riesgos del scraping

El scraping es una técnica potente pero tiene varios contras:

* Implica mayor tiempo de desarrollo y mayor esfuerzo en la limpieza de datos (en comparación con otras fuentes como APIs, BDs, ...)
* Si hay que scrapear gran cantidad de páginas, es lento
* Los servidores objetivo de nuestro scraping pueden tener técnicas para evitarlo. Por ejemplo, bloquear la IP temporalmente o introducir delays en las respuestas si hacemos muchas peticiones en poco tiempo. Esto pasa especialmente en las grandes webs recelosas de sus datos (p.e. linkedin, amazon, ...).
* El código de scraping escrito hoy puede no funcionar mañana, si la web destino cambia nombres, etiquetas o estructura. Si se sube a producción para lanzarlo periódicamente, hay que ser conscientes de que en algún momento fallará, y establecer mecanismos de alerta

## Javascript

Es posible que te encuentres con algún caso en el que no puedas descargar tal cual el html y parsearlo, principalmente por dos motivos:

* La estructura de la página se genera parcial o totalmente en cliente
* Debemos interactuar con algún elemento para mostrar la información que queremos (p.e. completar un campo de búsuqeda, hacer click en algún botón, ...)

En estos casos, hay que ejecutar en un navegador local el código javascript de la página destino. Para esta tarea, puedes utilizar [Selenium]().

[Aquí](https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251) un post con un ejemplo de uso.